In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # Importa la clase By
from time import sleep
import random
from selenium.webdriver.common.keys import Keys


In [2]:
df = pd.read_excel('C:/Users/9988/Desktop/cartas.xlsx', header=0)

In [3]:
options = Options()
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(options=options)
driver.get("https://www.google.com/")

In [4]:

# Imprime los nombres de las columnas para verificar
print("Nombres de las columnas:", df.columns)

Nombres de las columnas: Index(['Carta', 'PrecioCLP'], dtype='object')


In [5]:
cartas_diferencia_precio = []

In [6]:
# Revisa si los nombres de las columnas son correctos
if 'Carta' in df.columns and 'PrecioCLP' in df.columns:
    for index, row in df.iterrows():
        cartaquetenemos = row['Carta']
        preciodelacartaenCLP = row['PrecioCLP']
        preciodelacartaenUSD = preciodelacartaenCLP / 1000  # Conversión a USD

        # Busca la carta en el navegador
        username_input = driver.find_element(By.XPATH, '//*[@id="APjFqb"]')
        username_input.send_keys(cartaquetenemos + " starcitygames" + Keys.RETURN)
        sleep(5)

        primeraopcion = driver.find_element(By.XPATH, '//h3[contains(@class, "LC20lb MBeuO DKV0Md")]')
        primeraopcion.click()
        sleep(5)

        # Extrae el texto del elemento y elimina el símbolo $
        preciocartagringos_text = driver.find_element(By.XPATH, '//span[contains(@class, "price price--withoutTax")]').text.replace("$", "")

        # Convierte el texto a float
        preciocartagringos_float = float(preciocartagringos_text)

        # Compara los precios
        diferencia = preciocartagringos_float - preciodelacartaenUSD

        if diferencia == 0:
            mensaje = "Las cartas valen lo mismo, seguimos con la siguiente carta"
        else:
            if diferencia > 0:
                mensaje = f"La carta es más cara ahora, +{diferencia:.2f} USD"
            else:
                mensaje = f"La carta es más barata ahora, {diferencia:.2f} USD"

            cartas_diferencia_precio.append((cartaquetenemos, mensaje))
            print(mensaje)

        # Actualiza el DataFrame con el precio en USD y la diferencia
        df.at[index, 'PrecioUSD'] = preciodelacartaenUSD
        df.at[index, 'Diferencia'] = mensaje

        # Navega a Google para seguir con la siguiente carta
        driver.get("https://www.google.com")
        sleep(5)  # Asegúrate de que la página se carga correctamente





# Cierra el navegador al final
driver.quit()

# Guarda los resultados actualizados en un nuevo archivo Excel
df.to_excel('C:/Users/9988/Desktop/cartas_actualizadas.xlsx', index=False)

# Exporta la lista de cartas con diferencia de precio a un archivo de texto
with open('C:/Users/9988/Desktop/diferencia_precio_cartas.txt', 'w') as file:
    for carta, mensaje in cartas_diferencia_precio:
        file.write(f"{carta}: {mensaje}\n")


La carta es más cara ahora, +5.00 USD
La carta es más barata ahora, -0.24 USD
